In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')

In [2]:
features = pd.read_csv('data/features.csv')

In [3]:
unique_features = features.feature_text.unique()
unique_features.shape

(131,)

In [4]:
case_num = []
feature_num = []
unique_feature_num = []

for i, feature in enumerate(unique_features):
    case_num = case_num + features.loc[features['feature_text'] == feature, 'case_num'].to_list()
    feature_num = feature_num + features.loc[features['feature_text'] == feature, 'feature_num'].to_list()
    unique_feature_num = unique_feature_num + list(np.full(len(case_num) - len(unique_feature_num), i))

feature_mapping = pd.DataFrame({'case_num': case_num, 'feature_num': feature_num, 'unique_feature_num': unique_feature_num})


In [5]:
train_data = pd.read_csv('data/train.csv')

train_data['unique_feature_num'] = train_data.apply(lambda raw: feature_mapping.loc[(feature_mapping['case_num'] == raw['case_num']) & (feature_mapping['feature_num'] == raw['feature_num']), ['unique_feature_num']], axis = 1)
train_data.location.replace('[]', None, inplace=True)
train_data.dropna(inplace = True, axis = 0)

In [6]:
patient_notes = pd.read_csv('data/patient_notes.csv')

In [9]:
patient_notes_train_data = patient_notes.loc[patient_notes['pn_num'].isin(train_data.pn_num), :].copy()
patient_notes_train_data['features'] = patient_notes_train_data.apply(lambda raw: train_data.loc[train_data['pn_num'] == raw['pn_num'], 'unique_feature_num'].to_list(), axis = 1)